In [37]:
import sys
import pandas as pd
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
from pyspark.sql import SparkSession, functions, types, Row
spark = SparkSession.builder.appName('OSM point of interest extracter').getOrCreate()
assert spark.version >= '2.4' # make sure we have Spark 2.4+
spark.sparkContext.setLogLevel('WARN')
sc = spark.sparkContext
spark.conf.set("spark.sql.session.timeZone", "UTC")

In [47]:
df = pd.read_json('amenities-vancouver.json.gz', lines=True)
named_loc_df = df.dropna(axis=0)
named_loc_df.sort_values(by='timestamp', ascending=False)

,lat,lon,timestamp,amenity,name,tags
3223,49.237444,-123.032133,2020-06-28T18:09:16.000-07:00,clinic,Evergreen Community Health Centre,"{'addr:housenumber': '3425', 'website': 'http:..."
14573,49.119211,-122.671040,2020-06-28T18:09:16.000-07:00,veterinary,Apex,{'phone': '+1-604-514-1444'}
8843,49.156596,-122.913408,2020-06-28T06:48:09.000-07:00,fast_food,Apollo Pizza + Pasta,{'cuisine': 'pizza'}
15797,49.156596,-122.913753,2020-06-28T06:48:09.000-07:00,restaurant,Orange Blossom Chinese Food,{'cuisine': 'chinese'}
15785,49.105144,-122.690461,2020-06-28T05:51:43.000-07:00,charging_station,ChargePoint,"{'brand:wikidata': 'Q5176149', 'brand': 'Charg..."
...,...,...,...,...,...,...
26,49.345193,-123.149799,2008-11-14T06:03:08.000-08:00,school,Sentinel Secondary School,{'created_by': 'Potlatch 0.10f'}
25,49.327883,-123.151945,2008-11-14T05:58:14.000-08:00,fuel,Shell,{'created_by': 'Potlatch 0.10f'}
24,49.327947,-123.156395,2008-11-10T05:46:54.000-08:00,fuel,Petro Canada,{'created_by': 'Potlatch 0.10f'}
46,49.341975,-123.063321,2008-04-28T02:25:55.000-07:00,school,Carisbrook Elementary,{'created_by': 'Potlatch 0.8b'}


In [40]:
from collections import Counter
amenities_list = set(named_loc_df['amenity'])
counter_list = Counter(list(named_loc_df['amenity']))

In [41]:
counter_list

Counter({'cafe': 1055,
         'fast_food': 1131,
         'place_of_worship': 96,
         'fuel': 188,
         'restaurant': 2444,
         'parking_entrance': 5,
         'pub': 186,
         'school': 62,
         'community_centre': 58,
         'pharmacy': 304,
         'dentist': 226,
         'doctors': 99,
         'post_office': 187,
         'childcare': 43,
         'public_building': 1,
         'bank': 458,
         'cinema': 12,
         'theatre': 41,
         'ferry_terminal': 28,
         'bar': 173,
         'library': 50,
         'car_rental': 33,
         'bicycle_rental': 201,
         'clinic': 114,
         'public_bookcase': 38,
         'university': 6,
         'dojo': 13,
         'toilets': 18,
         'arts_centre': 11,
         'ice_cream': 71,
         'bench': 95,
         'shelter': 11,
         'bicycle_parking': 3,
         'recycling': 31,
         'drinking_water': 9,
         'fountain': 7,
         'veterinary': 68,
         'bureau_de_change